In [1]:
import urllib.request
import json
import pandas as pd
import bs4


In [20]:
response = urllib.request.urlopen('http://www.burgerking.co.kr/api/store/searchmap/empty/?areacd=')
bgk_data = json.loads(response.read().decode('utf-8'))
bgk_tbl = pd.DataFrame(bgk_data)
bgk_tbl.head()

,AllHour,CloseCleaning,ClosePeakSeason,CloseWeekday,Delivery,DriveThrough,Morning,NewAddr,NewAddr2,OpenTime,PhoneNumber,PointX,PointY,StoreNM,StoreSQ
0,N,,,22:00,Y,N,N,서울특별시 금천구 가산디지털1로,168 우림라이온스밸리 A동,09:00,02-853-0332,37.480896,126.882133,가산디지털점,1
1,N,,,22:00,Y,N,Y,서울특별시 금천구 디지털로9길 46,이앤씨드림타워7차 101호,08:00,070-7438-8601,37.480247,126.886978,가산이앤씨7점,325
2,N,,/ 토: 10:00 ~ 23:00 / 일: 10:00 ~ 22:00,22:00 / 금: 11:00~23:00,N,N,N,서울특별시 금천구 디지털로10길,9 HYHILL 6층,월~목: 11:00,02-2136-9962,37.477620,126.889053,가산현대시티아울렛점,2
3,N,,,2018.06.29(금),N,N,N,서울특별시 강서구 양천로,559 이마트3층 (가양동),휴점기간 : 2018.05.27(일),02-3664-0221,37.558193,126.861816,가양이마트점,3
4,N,,,24:00,Y,N,N,서울특별시 서대문구 북가좌동,487 성공타워 1층,09:00,02-303-0392,37.570650,126.912750,가재울뉴타운점,275


In [21]:
bgk_locs = pd.DataFrame(bgk_tbl['NewAddr'].apply(lambda v: v.split()[:2]).tolist(),
                        columns=('d1', 'd2'))
bgk_locs.head()

,d1,d2
0,서울특별시,금천구
1,서울특별시,금천구
2,서울특별시,금천구
3,서울특별시,강서구
4,서울특별시,서대문구


In [22]:
bgk_locs['d1'].unique()

array(['서울특별시', '서울시', '경기도', '인천시', '경상남도', '인천광역시', '강원도', '충청남도',
       '세종특별자치시', '대전광역시', '대전시', '충청북도', '부산광역시', '부산', '울산광역시', '대구광역시',
       '경상북도', '경남', '전라남도', '광주광역시', '전라북도', '제주특별자치도'], dtype=object)

In [23]:
bgk_locs['d2'].unique()

array(['금천구', '강서구', '서대문구', '서초구', '강남구', '구로구', '송파구', '광진구', '성북구',
       '마포구', '노원구', '관악구', '용산구', '강동구', '동작구', '영등포구', '중구', '중랑구',
       '양천구', '강북구', '은평구', '안산시', '도봉구', '성동구', '서구', '동대문구', '종로구',
       '진주시', '양평군', '고양시', '과천시', '수원시', '광명시', '광주시', '구리시', '파주시',
       '김포시', '남양주시', '동두천시', '화성시', '부천시', '성남시', '군포시', '용인시', '시흥시',
       '안양시', '오산시', '의정부시', '이천시', '평택시', '하남시', '남구', '부평구', '송도국제대로',
       '계양구', '남동구', '연수구', '강릉시', '홍천군', '속초시', '원주시', '춘천시', '당진시',
       '서산시', '천안시', '보듬5로', '홍성군', '유성구', '동구', '청주시', '청원군', '진천군',
       '충주시', None, '사하구', '금정구', '사상구', '해운대구', '동래구', '부산진구', '연제구',
       '기장군', '수성구', '북구', '달서구', '달성군', '안동시', '상주시', '영주시', '경산시', '경주시',
       '구미시', '김천시', '포항시', '거제시', '창원시', '사천시', '김해시', '광양시', '나주시',
       '목포시', '순천시', '여수시', '무안군', '광산구', '군산시', '익산시', '전주시', '정읍시', '제주시'], dtype=object)

In [7]:
d1_aliases = """서울시:서울특별시 충남:충청남도 강원:강원도 경기:경기도 충북:충청북도 경남:경상남도 경북:경상북도
전남:전라남도 전북:전라북도 제주도:제주특별자치도 제주:제주특별자치도 대전시:대전광역시 대구시:대구광역시 인천시:인천광역시
광주시:광주광역시 울산시:울산광역시"""
d1_aliases = dict(aliasset.split(':') for aliasset in d1_aliases.split())
bgk_locs['d1'] = bgk_locs['d1'].apply(lambda v: d1_aliases.get(v,v))

In [13]:
bgk_locs['d1'].unique()

array(['서울특별시', '경기도', '인천광역시', '경상남도', '강원도', '충청남도', '세종특별자치시', '대전광역시',
       '충청북도', '부산광역시', '부산', '울산광역시', '대구광역시', '경상북도', '전라남도', '광주광역시',
       '전라북도', '제주특별자치도'], dtype=object)

In [24]:
bgk_locs[bgk_locs['d1'] == '수원시']

,d1,d2


In [25]:
bgk_locs.iloc[101] = ['경기도', '수원시']

In [26]:
bgk_locs['d2'].unique()

array(['금천구', '강서구', '서대문구', '서초구', '강남구', '구로구', '송파구', '광진구', '성북구',
       '마포구', '노원구', '관악구', '용산구', '강동구', '동작구', '영등포구', '중구', '중랑구',
       '양천구', '강북구', '은평구', '안산시', '도봉구', '성동구', '서구', '동대문구', '종로구',
       '진주시', '양평군', '고양시', '수원시', '광명시', '광주시', '구리시', '파주시', '김포시',
       '남양주시', '동두천시', '화성시', '부천시', '성남시', '군포시', '용인시', '시흥시', '안양시',
       '오산시', '의정부시', '이천시', '평택시', '하남시', '남구', '부평구', '송도국제대로', '계양구',
       '남동구', '연수구', '강릉시', '홍천군', '속초시', '원주시', '춘천시', '당진시', '서산시',
       '천안시', '보듬5로', '홍성군', '유성구', '동구', '청주시', '청원군', '진천군', '충주시', None,
       '사하구', '금정구', '사상구', '해운대구', '동래구', '부산진구', '연제구', '기장군', '수성구',
       '북구', '달서구', '달성군', '안동시', '상주시', '영주시', '경산시', '경주시', '구미시', '김천시',
       '포항시', '거제시', '창원시', '사천시', '김해시', '광양시', '나주시', '목포시', '순천시',
       '여수시', '무안군', '광산구', '군산시', '익산시', '전주시', '정읍시', '제주시'], dtype=object)

In [27]:
B = bgk_locs.apply(lambda r: r['d1'] + ' ' + r['d2'], axis=1).value_counts()
B.head()

TypeError: ('must be str, not NoneType', 'occurred at index 218')